## Spark/pyspark for "big" data 

This notebook provides a demonstration of working with gzipped (compressed) JSON-L (line-delineated Javascript Object Notation) files on single computer using Python, pyspark, and Apache Spark.

#### Specs
 - Python version: 3.8
 - Spark version: 3.1.2
 - Java version: OpenJDK 11.0.11+9 
 - MacBook Pro, 2.8 GHz Intel Core i7
 - Number of cores: 4
 - Memory: 16 GB
 - OS: macOS Mojave (10.14.6)
 
#### Setup
 - 26GB of Tweets (gzipped JSON-L format)
 - 1,000 files
 - Files stored locally
 - Spark running in "local" mode (i.e., on a single machine)

#### Spark/Java system settings

Relevant lines from `.bash_profile` in my home directory. (These values will differ, depending on your setup.)

```
export SPARK_HOME=/opt/spark
export JAVA_HOME=/Library/Java/JavaVirtualMachines/jdk-11.0.11+9/Contents/Home
export PATH=$SPARK_HOME/bin:$PATH


```

#### A "naive" Python approach 

 - Loop over a list of gzipped JSON-L files
 - Increment a counter
 - Returns the number of documents across all files 
 - Avoids reading all files into memory at once
 - Takes a _long_ time to run

In [ ]:
import gzip
import glob
import json

def count_tweets(files):
    i = 0
    for file in files:
        with gzip.open(file, mode='rt') as f:
            for line in f:
                i += 1
    return i

 - Get a list of files we want to read

In [ ]:
files = glob.glob('/Users/dsmith/Documents/code/workshops/gwlibraries-workshops/spark-gwcoders/coronavirus/*.json.gz')

 - Takes 11 minutes to process 1,000 files

In [ ]:
%time count_tweets(files)

What we might want is something like this:

,text,max(retweet_count)
0,RT @NicholsUprising: Sicily has figured out th...,211408
1,RT @_theghettomonk: Not enough praise has gone...,209404
2,RT @_theghettomonk: Not enough praise has gone...,209248
3,RT @MMarieSophie: HIS NAME IS STEPHEN WAMUKOTA,201466
4,RT @MMarieSophie: HIS NAME IS STEPHEN WAMUKOTA...,201304
...,...,...
95,RT @RachelPatzerPhD: My spouse is a physician ...,59764
96,RT @LeoSaldanaP: #quedateEnTuCasa Soy neumólog...,59723
97,RT @anmmoo: [ฝากรีนะคะ] พ่อเราและเพื่อนๆเขา ทำ...,59677
98,RT @DrPeckPNP: This is how far back we have to...,59458


### Leveraging Spark

#### Preliminaries

 - `findspark` connects our notebook to the pyspark library

In [ ]:
import findspark
findspark.init()

Importing some necessary tools
- A `SparkSession` instance provides our interface with the Spark runtime
- Spark comes with a number of functions for working with data modeled on SQL

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

 - Create a session 
 - You might encounter an error if you haven't installed Spark/Java correctly

In [ ]:
spark = SparkSession.builder.appName('GWCoders-Demo').getOrCreate()

#### Understanding Spark's data model

 - Works well with structured data (JSON, CSV, etc.)
 - Strongly typed: each field needs a well-defined (Scala) data type
 - Can infer the **schema** instead of our having to define it explicitly

##### A scaleable approach
 - If your dataset uses the same fields consistently, read a small sample first
 - Reuse the schema when reading the entire dataset
 - `read.json`: use with a single file or a whole directory
 - Add the prefix `file://` to direct Spark to use your local filesystem


In [ ]:
sample = spark.read.json('file://' + files[0])

##### DataFrames in Spark
 - "Lazily" evaluated: data only read into memory at the time of computation
 - The DataFrame provides a "frame" with type information that is filled as needed
 - We can use the `show` method to view the first N rows
 - A Spark DataFrame supports nested data types
 - Use the `printSchema` method to view the schema hierarchy
 - Use the `schema` property to acess the schema directly

In [ ]:
sample.show()

In [ ]:
sample.printSchema()

In [ ]:
# We'll re-use this later on
schema = sample.schema

#### Spark DataFrame methods
 - Modeled on SQL
 - Use the `select` method to create a subset of one or more columns
 - Use the `where` or `filter` functions to create a subset of rows matching certain conditions
 - The `pyspark.sql.functions` [module](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html) provides access to Spark's DataFrame functions for manipulating data 

In [ ]:
sample.count()

In [ ]:
# The id_str is the field in a Tweet document that uniquely identifies the Tweet
sample.select('id_str').distinct().count()

In [ ]:
# This result is the same as the above
# Here we create a new DataFrame with a single column containing our distinct count
sample.select(F.countDistinct('id_str')).show()

In [ ]:
# We can filter our sample to just those Tweets that are re-tweets
sample.where(F.col('retweeted_status').isNotNull()).select(F.countDistinct('id_str')).show()

##### Grouping and aggregating over fields
 - The `groupBy` methods works like the `groupby` method in pandas
 - Select the relevant columns, group, then apply an aggregation
 - Need to call `show` to display the result

In [ ]:
sample.select('lang', 'id_str').groupBy('lang').agg(F.countDistinct('id_str')).show()

##### Working with nested data
 - Much harder to do in pandas
 - Use **dot notation** to drill down into a hiearchical column
 - The `explode` method: unroll an array into a new column, creating 1 row per element 
 - Chain `select` statements to extract subfields from array elements
 - Create intermediate DataFrames for group by/aggregate operations
 - "Lazy" evaluation: not computation performed until `show` is called

In [ ]:
# The hashtags field is an array nested under the entities field
sample.select('entities.hashtags').show()

In [ ]:
# It consists of multiple subfields
# The text subfield contains the hashtag text
sample.select(F.explode('entities.hashtags'), 'id_str').show()

In [ ]:
# the take method makes more explicit how the data is structured in each row
sample.select(F.explode('entities.hashtags'), 'id_str').take(1)

In [ ]:
# We can refer to the _exploded_ hashtags field by its default name, col
sample.select(F.explode('entities.hashtags'),'id_str')\
        .select('id_str', 'col.text').show()

In [ ]:
hashtags = sample.select(F.explode('entities.hashtags'), 'id_str')\
        .select('id_str', 'col.text')

In [ ]:
# Running the groupBy method on our intermediate DataFrame
hashtags.groupBy('text').count().sort(F.desc('count')).show()

 - We can use nested fields in other kinds of aggregations, too.

In [ ]:
# None of the Tweets in our sample have been retweeted
sample.select(F.max('retweet_count')).show()

In [ ]:
# The retweeted_status field contains metadata about the Tweet that was retweeted
# We can use that field to find the most commonly retweeted Tweets in our set
sample.select('retweeted_status.retweet_count', 'text').groupby('text').max('retweet_count')\
        .sort(F.desc('max(retweet_count)')).show()

#### Spark SQL
 - DataFrame functions can be expressed as [direct SQL expressions](https://spark.apache.org/docs/latest/api/sql/index.html)
 - Create a temp view/table 
 - Run SQL statements against the temp view
 - Handy for complex queries

In [ ]:
# Defines a SQL statement as a Python multi-line string
lang_counts = '''
select lang, count(distinct id_str) as tweet_count
from tweets
group by lang
order by tweet_count desc
'''

In [ ]:
# Instantiates a temporary view (referenced in our SQL expression)
sample.createOrReplaceTempView('tweets')

In [ ]:
# Use the spark.sql method to execute the query
spark.sql(lang_counts).show()

Below is a SQL query that will create a table of edges between Twitter users who have authored Tweets in our dataset and Twitter users mentioned by those authors.

In [ ]:
# Using a nested query to explode the mentions field, which is an array, each element of which
# corresponds to a mentioned user
mentions_sql = '''
select mentions.screen_name as mentioned_screen_name,
mentions.id_str as mentioned_user_id,
user_screen_name,
user_id
from (select explode(entities.user_mentions) as mentions,
    user.id_str as user_id,
    user.screen_name as user_screen_name
    from tweets)
'''

In [ ]:
sample.createOrReplaceTempView('tweets')

In [ ]:
spark.sql(mentions_sql).show()

#### Querying the full dataset
 - From an in-memory sample to a full dataset on disk
 - Query optimization: write/show only at the end
 - Grouping/joining data takes longer other kinds of transformations
 - Re-use the schema wherever possible
 - The Spark [Jobs UI](http://localhost:4040/jobs/) can help you monitor/troubleshoot


In [ ]:
# Reading in our full dataset
# Note the call to the schema method, using the variable defined earlier
df = spark.read.schema(schema).json('file:///Users/dsmith/Documents/code/workshops/gwlibraries-workshops/spark-gwcoders/coronavirus/')

In [ ]:
# Create a new temp view in order to apply our SQL
df.createOrReplaceTempView('tweets')
results = spark.sql(mentions_sql)

In [ ]:
# Took 6.9 minutes on my machine
results.write.csv('file:///Users/dsmith/Documents/code/workshops/gwlibraries-workshops/spark-gwcoders/covid-mentions')

In [ ]:
# Creating a DataFrame of the top 100 most retweeted Tweets across the whole datasset
# http://localhost:4040/jobs/
retweets = df.select('retweeted_status.retweet_count', 'text').groupby('text').max('retweet_count')\
        .sort(F.desc('max(retweet_count)'))

In [ ]:
retweets.limit(100).toPandas()

#### Take-aways & considerations
 - Spark is useful for
   - Datasets too large to fit into memory
   - Deeply nested data structures
   - Distributed datasets
 - Requires Java & Scala
 - Interfaces exist for Python (PySpark) and [R](https://spark.apache.org/docs/latest/sparkr.html)
 - Type & organization of the data matter (e.g., many small files vs. a few large files)
 - Many [configuration](https://spark.apache.org/docs/latest/configuration.html) options 
 - Groupby aggregations & joins "cost" more than other transformations

#### Further resources

##### Installation guides consulted

   - https://kevinvecmanis.io/python/pyspark/install/2019/05/31/Installing-Apache-Spark.html
   - https://medium.com/swlh/pyspark-on-macos-installation-and-use-31f84ca61400
   - https://www.datacamp.com/community/tutorials/installation-of-pyspark

Note that some guides recommend installing Scala separately. If you do that, make sure that the version of Scala you are using is compatible with the version of Spark you are using. More information on the [Apache Spark downloads](https://spark.apache.org/downloads.html) page.

##### Books & tutorials on Spark/pyspark

   - [Spark: The Definitive Guide](https://learning.oreilly.com/library/view/spark-the-definitive/9781491912201/)
   - [Data Analytics with Spark Using Python](https://learning.oreilly.com/library/view/data-analytics-with/9780134844855/)
   - [Learning Spark](https://learning.oreilly.com/library/view/learning-spark-2nd/9781492050032/)
   - [Best Practices for Spark Performance Optimization](https://developer.ibm.com/blogs/spark-performance-optimization-guidelines/)
   